# Second model 

This model will try to predict wether pe is present or not on a full image (RGB level)

## Imports

In [1]:
import numpy as np
from balance import *
from model import *
from torchsummary import summary

In [2]:
import torch
torch.cuda.empty_cache()

print(f'PyTorch version: {torch.__version__}')
print("GPU found :)" if torch.cuda.is_available() else "No GPU :(")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

# Load datasets

In [3]:
full_image_dataset = torch.load('datasets/full_image_dataset.pt')

## Simple CNN from scratch

In [4]:
class ModelV2(nn.Module):
    
    def __init__(self, input_channels):
        """Simple Convnet with 2 convolution layer + pooling, with 2 fully connected at the end"""
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.fc1 = nn.Linear(32*62*62 , 120)
        self.fc2 = nn.Linear(120, 1)
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32*62*62) # Flatten
        x = F.relu(self.fc1(x))
        
        # Binary classification
        out = self.sigmoid(self.fc2(x))
        
        return out

In [5]:
model = ModelV2(3).to(device)
summary(model, (3, 256, 256))

## Split data into train and test

In [6]:
batch_size = 126
ratio=0.2

train_loader, test_loader = train_test_split(full_image_dataset, ratio, batch_size)

## Model Training

In [7]:
num_epoch = 30
criterion = nn.BCELoss()
optimizer=optim.Adam(model.parameters(), lr=0.001)
save_filename = './models/model_v2.h5'

model, losses = train(model, num_epoch, criterion, train_loader, optimizer, device, save_filename)

In [8]:
plot_loss(losses, num_epoch, len(train_loader), 'Loss model v2 full image.png')

## Test the model

In [9]:
# Load model for evaluation
trained_model = ModelV2(input_channels=3)
trained_model.load_state_dict(torch.load('./models/model_v2.h5'))
trained_model.to(device)

ModelV2(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=123008, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [10]:
accuracy, total = evaluate(trained_model, test_loader, device)

In [11]:
total = len(test_loader)
print(f'Accuracy of the network on the {total} test images: {accuracy:.2f}')